# 2.1 마케팅 문구 생성 프롬프트 엔지니어링

이번 챕터에서는 마케팅 문구를 생성하기 위한 프롬프트를 작성해보겠습니다.

## chatGPT API 사용 설정

In [2]:
import openai
import os

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [10]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "안녕하세요!"}
    ]
)

In [11]:
response["choices"][0]["message"]["content"]

'안녕하세요! 어떻게 도와드릴까요?'

## 프롬프트 엔지니어링

### AI에게 역할부여

광고 문구를 생성하기 위해서 먼저 system message를 이용해서 AI에게 "전문 카피라이터" 라는 역할을 부여하겠습니다.  
그 다음, 유저로부터 프롬프트만 전달받아서 응답을 리턴하도록 함수화 하겠습니다.

In [44]:
def request_chat_completion(prompt, stream=False):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 전문 카피라이터입니다."},
            {"role": "user", "content": prompt}
        ],
        stream=stream
    )
    return response

"카누" 라는 커피 브랜드의 광고 문구를 작성해달라는 간단한 프롬프트를 넣어서 텍스트를 생성해보겠습니다.

In [19]:
prompt = "'카누'라는 커피믹스 브랜드의 SNS 마케팅을 위한 카피를 3개 생성해줘"
response = request_chat_completion(prompt)

In [20]:
generated_text = response["choices"][0]["message"]["content"]

In [21]:
print(generated_text)

1. "매일 아침, 향긋한 커피향에 일어나는 여운을 느껴보세요. 카누의 커피믹스는 당신의 아침을 완벽하게 만들어줄 것입니다. 풍부한 맛과 향을 가진 카누로 당신의 하루를 시작해보세요!"

2. "커피를 즐기는 새로운 방법을 찾고 있다면, 카누의 다양한 커피믹스를 만나보세요. 높은 품질의 커피 원두로 만들어진 카누는 아늑한 가정에서도 프로의 맛을 즐길 수 있는 최고의 선택입니다. 당신의 커피 경험을 한 단계 업그레이드 해보세요!"

3. "여러분, 카누는 커피에 대한 열정과 소중한 시간을 지닌 분들을 위해 준비되었습니다. 카누의 커피믹스는 세련된 맛과 향을 가지고 있어, 당신의 커피 탐험을 도와줄 것입니다. 매일의 일상에서도 특별함을 느끼고 싶다면, 카누가 함께할 거예요!"


결과를 보면 마케팅 문구를 만들어주긴 하지만 지나치게 길고, 재치가 느껴지지 않습니다.  
프롬프트 엔지니어링을 이용해서 우리가 원하는 대로 텍스트를 생성하도록 제어해보겠습니다.

### 프롬프트 생성

마케팅 문구를 생성하고자 하는 제품명과 제품에 대한 설명을 파라미터로 전달받아서 프롬프트를 생성해보겠습니다.  
이처럼 파라미터를 전달받는 경우, AI에게 먼저 수행해야할 지시를 알려주고 구분자를 사용해서 그 사이에 파라미터를 지정해주는 것이 좋습니다.  
지시 다음 부분에 최대 길이나 어떤 느낌이 나면 좋겠다는 걸 적어주세요.

In [32]:
def generate_prompt(product_name, product_desc):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
---
제품명: {product_name}
제품설명: {product_desc}
---
""".strip()
    return prompt

In [33]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스"
)

In [34]:
print(prompt)

제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
---
제품명: 카누
제품설명: 집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스
---


In [35]:
response = request_chat_completion(prompt)
generated_text = response["choices"][0]["message"]["content"]
print(generated_text)

"집에서도 찰떡같은 아메리카노 맛! #카누로 풍요로운 커피 라이프를 즐겨보세요. ☕️✨"


### 생성하는 텍스트 길이, 개수 조정

이번에는 생성하는 텍스트의 길이와 개수를 프롬프트로 조정해보겠습니다.  
generate_prompt 함수에 파라미터를 추가해보겠습니다.  
개수의 경우 문구를 n개 생성해달라고 적어주면 잘 반영됩니다.  
텍스트 길이 조정의 경우엔 몇 자 이내로 적어달라는 것 보다는, 몇 단어 이내로 적어달라는 것이 좋으며, 앞에 "반드시"를 붙여주세요.

In [36]:
def generate_prompt(product_name, product_desc, num, max_length):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 {num}개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 {max_length} 단어 이내로 작성해주세요.
---
제품명: {product_name}
제품설명: {product_desc}
---
""".strip()
    return prompt

In [37]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스",
    num=5,
    max_length=7
)

In [39]:
print(prompt)

제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 5개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 7 단어 이내로 작성해주세요.
---
제품명: 카누
제품설명: 집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스
---


In [40]:
response = request_chat_completion(prompt)
generated_text = response["choices"][0]["message"]["content"]
print(generated_text)

1. 카누와 함께라면 집에서도 완벽한 아메리카노!
2. 커피 믹스의 새로운 스탠더드, 카누!
3. 무한한 커피 탐험, 카누와 함께하세요.
4. 집에서도 카페처럼 향기로운 아메리카노, 카누.
5. 손쉽게 즐기는 카페 퀄리티, 카누의 맛에 빠져보세요.


### 키워드 포함

마케팅 문구에 반드시 포함했으면 하는 키워드가 있다면 프롬프트에 포함할 수 있습니다.  
generate_prompt에 파라미터를 추가하고, 프롬프트를 약간 수정해보겠습니다.

In [41]:
def generate_prompt(product_name, product_desc, num, max_length, keywords):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 {num}개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 {max_length} 단어 이내로 작성해주세요.
키워드가 주어질 경우, 반드시 키워드 중 하나를 포함해야 합니다.
---
제품명: {product_name}
제품설명: {product_desc}
키워드: {keywords}
---
""".strip()
    return prompt

In [46]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스",
    num=5,
    max_length=7,
    keywords=["브라질", "풍미", "향기"]
)

In [47]:
response = request_chat_completion(prompt)
generated_text = response["choices"][0]["message"]["content"]
print(generated_text)

1. 집에서도 브라질의 풍미와 향기를 즐겨보세요!
2. 아메리카노 맛으로 떠나는 특별한 커피 여행!
3. 집에서도 카페처럼! 향긋한 아메리카노 한 잔.
4. 커피의 맛과 향기, 집에서 만나는 새로운 경험.
5. 특별한 커피 믹스로 집에서 브라질의 풍미를 만나세요!


## 스트리밍 처리

마지막으로 스트리밍 방식으로 텍스트 생성을 요청한 다음, 결과를 화면에 출력해보겠습니다.

In [45]:
response = request_chat_completion(prompt=prompt, stream=True)
for chunk in response:
    delta = chunk.choices[0]["delta"]
    if "content" in delta:
        print(delta["content"], end="")
    else:
        break

1. "강렬한 브라질 풍미, 즐기는 순간 풍성한 향기"
2. "카페보다 더 풍미로운 집에서의 아메리카노"
3. "브라질 향기를 집에서 느껴보세요"
4. "집에서 편안하게, 그리고 맛있게 즐기는 커피"
5. "집에서도 카페의 맛과 향을 담은 최고의 선택"

## 정리

지금까지 마케팅 문구를 생성하기 위한 프롬프트 엔지니어링을 배워봤습니다.  
어떤가요? 굉장히 쉽지 않나요? 내가 결과로 얻고 싶은 텍스트를 자연어로 묘사하고, 파라미터만 쏙쏙 넣어주면 됩니다.  
이제 streamlit을 이용해서 UI를 개발하고, 여기에 마케팅 문구 생성 기능을 적용하겠습니다.